In [1]:
# run once
# ! pip install gspread oauth2client df2gspread boto3
from dcicutils import ff_utils
from functions.notebook_functions import *
import pandas as pd

my_key = get_key('default')

all_users = ff_utils.search_metadata('/search/?type=User', key = my_key)
all_labs = ff_utils.search_metadata('/search/?type=Lab', key = my_key)
print(len(all_users))
print(len(all_labs))
# all_labs = 
# all_awards = 

524
95


In [8]:
#print(all_labs[0])

In [3]:

def compare(row, columns, all_users, all_labs,results_list):
    extra = ""
    second_mail = ""
    delete = False
    first_name = row['First Name'].strip()
    last_name = row['Last Name'].strip()
    mail = row['Email'].strip().lower()
    role = row['Role'].strip()
    lab_id = 'None'
    users_list = []
    
    
    
    if not first_name:
        return
    try:
        contact_pi = row['PI Contact']
    except:
        contact_pi = row['Contact PI']
        
    #Find the lab corresponding to the PI
    for i in all_labs:
        if contact_pi in i['display_title']:
            lab_id = i['@id']
            award = i['display_title']
      
        
    if "Additional email" in columns:
        second_mail = row['Additional email']
    
    for val in row:
        if 'delete' in val.lower():
            delete = True
        elif 'remove' in val.lower():
            delete = True
    
    # Check for first name last name combination
    res1 = [i for i in all_users if i['first_name'] == first_name if i['last_name'] == last_name]
    if res1:
        
        if delete:
            print(mail, 'user exists but tagged with delete')
        return True
    
    # Are the name surname reversed
    res2 = [i for i in all_users if i['first_name'] == last_name if i['last_name'] == first_name]
    if res2:
        print(mail, 'user has reversed name surname')
        if delete:
            print(mail, 'user exists but tagged with delete')
        return True
    
    # Do we have the email 
    res3 = [i for i in all_users if i['email'] == mail]
    if res3:
        #print(mail, 'found on our system, OH name is',first_name, last_name, 'DATA name is', res3[0]['first_name'], res3[0]['last_name'])
        if delete:
            print(mail, 'user exists but tagged with delete')
        return True
    
    if second_mail:
        res4 = [i for i in all_users if i['email'] == second_mail]
        if res4:
            #print(mail, 'found on our system, OH name is',first_name, last_name, 'DATA name is', res3[0]['first_name'], res3[0]['last_name'])
            if delete:
                print(mail, 'user exists but tagged with delete')
            return True
    
    # Do we have same surname
    res5 = [i for i in all_users if i['last_name'] == last_name]
    f_names = [i['first_name'] for i in res5]
    if res5:
        if delete:
            return
        print(mail, first_name, last_name, 'matches by surname to these names', ', '.join(f_names), 'PI: ', contact_pi,lab_id,role)
        users_list.append(mail)
        users_list.append(first_name)
        users_list.append(last_name)
        users_list.append(lab_id)
    
        users_list2 = '\t'.join(users_list)
        results_list.append(users_list2)
        return True
    
    # if we cant find but tagged with delete
    if delete:
        return
    
    if not mail:
        return
    
    
    print(first_name, last_name, mail, contact_pi,lab_id,role)
    
    #This creates a list of the missing users in the same format as the excel sheet for the item 'USER' so that it is easy to coopy and paste
    users_list.append(mail)
    users_list.append(first_name)
    users_list.append(last_name)
    users_list.append(lab_id)
    
    users_list2 = '\t'.join(users_list)

    results_list.append(users_list2)


        
    
            
            


In [4]:
# GET OH SHEET
import boto3
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import json

# GET KEY FROM S3 
s3 = boto3.resource('s3')
obj = s3.Object('elasticbeanstalk-fourfront-webprod-system', 'DCICjupgoogle.json')
cont = obj.get()['Body'].read().decode()
key_dict = json.loads(cont)
SCOPES = 'https://www.googleapis.com/auth/spreadsheets.readonly'
creds = ServiceAccountCredentials.from_json_keyfile_dict(key_dict, SCOPES)
gc = gspread.authorize(creds)

# The ID and range of a sample spreadsheet.
book_id = '13gFjHcYEwYL2Xpr9-IHtJNYYnYlAQVdyXvCUeSzla6w'
sheet_names = ['DCIC & OH', 'IT', 'NT', 'NOFIC', 'NBC','TCPA']
book = gc.open_by_key(book_id)


users_list = [] #This is the list of the users that are not in the portal, so we can copy and paste them in excel workbook

for name in sheet_names:
    print()
    print(name)

    worksheet = book.worksheet(name)
    table = worksheet.get_all_values()
    ##Convert table data into a dataframe
    df = pd.DataFrame(table[1:], columns=table[0])
    col_names = df.columns.values

    for index, row in df.iterrows():
        compare(row, col_names, all_users, all_labs,users_list)
    

            



DCIC & OH
joshhjang@gmail.com user exists but tagged with delete
ananda.datta@wustl.edu user exists but tagged with delete

IT
 Ruiwu Liu matches by surname to these names Jie, Tristin PI:  Kit Lam /labs/kit-lam-lab/ Co-I

NT

NOFIC
meredk@uw.edu user exists but tagged with delete
mgillies@uw.edu user exists but tagged with delete
 Karrie Brown matches by surname to these names Lewis PI:  Job Dekker/Leonid Mirny None 
 Hanjun Shin matches by surname to these names Yongdae PI:  Frank Alber /labs/frank-alber-lab/ 
 Xiulan Wang matches by surname to these names Xiaotao, Zhong, Yuchuan, Yanpeng, Su, Ping, Miao, Juan, Haifeng, Ting PI:  Frank Alber /labs/frank-alber-lab/ 

NBC

TCPA
brad.cairns@hci.utah.edu Brad Cairns matches by surname to these names Bradley PI:  Brad Cairns /labs/brad-cairns-lab/ PI
leonid@mit.edu user has reversed name surname
shendure@uw.edu Jay Shendure matches by surname to these names Jay Ashok PI:  Steve Henikoff /labs/steve-henikoff-lab/ Co-I


In [15]:
for i in users_list:
    print(i)

connorhorton@college.harvard.edu	Connor	Horton	/labs/peter-park-lab/
ihjacobo@eng.ucsd.edu	Irene Hom	Jacobo	/labs/sheng-zhong-lab/
ckling@ucsd.edu	Carol	Kling	/labs/sheng-zhong-lab/
kzaletar@eng.ucsd.edu	Kathia Zaleta	Rivera	/labs/sheng-zhong-lab/
wec125@eng.ucsd.edu	Weizhong	Chen	/labs/sheng-zhong-lab/
luhebert@eng.ucsd.edu	Lucie	Hebert	/labs/sheng-zhong-lab/
hzzheng@eng.ucsd.edu	Haizi	Zheng	/labs/sheng-zhong-lab/
wew245@eng.ucsd.edu	Weixin	Wu	/labs/sheng-zhong-lab/
car032@eng.ucsd.edu	Kate	Rivera	/labs/sheng-zhong-lab/
frank.chuang@cbst.ucdavis.edu	Frank	Chuang	/labs/kit-lam-lab/
	Ruiwu	Liu	/labs/kit-lam-lab/
dengw@berkeley.edu	Jorge	Deng	/labs/robert-singer-lab/
arobles77@berkeley.edu	Astou	Robles	/labs/robert-singer-lab/
frank.l.xie@berkeley.edu	Frank	Xie	/labs/robert-singer-lab/
maximiliaan.huisman@umassmed.edu	Maximillian	Huisman	/labs/david-grunwald-lab/
gcadwell@salk.edu	Greg	Cadwell	/labs/clodagh-o-shea-lab/
pdennett@salk.edu	Preston	Dennett	/labs/clodagh-o-shea-lab/
tgg@seas.